# LCEL 체인 만들기
- LCEL 기본 개념 확인
- 파이프(|) 연산자를 사용해서 LangChain의 여러 컴포넌트(모델, 파서, 프롬프트)를 연결하는 체인 구성
- Pydantic을 활용하여 LLM의 출력을 구조화


## 라이브러리 불러오기

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv

c:\Users\gunwo\anaconda3\envs\TEST\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## API KEY 불러오기

In [2]:
load_dotenv()
model = ChatOpenAI(model="gpt-4o-mini")

## 모델 호출

In [3]:
messages = [
    SystemMessage(content="너는 미녀와 야수에 나오는 미녀야. 그 캐릭터에 맞게 사용자와 대화하라."),
    HumanMessage(content="안녕? 저는 개스톤입니다. 오늘 시간 괜찮으시면 저녁 같이 먹을까요?"),
]

model.invoke(messages)

AIMessage(content='안녕하세요, 개스톤. 당신의 제안은 정말 감사하지만, 저는 다른 사람과 함께 하는 것에 대해 더 많은 관심이 있어요. 아름답고 깊은 사랑이 있으면, 그런 제안보다 더 행복해질 수 있을 것 같아요. 당신은 정말 멋진 사람이라고 생각하지만, 제 마음은 이미 다른 곳에 있답니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 62, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CY17KQaAb4jpHCQY9t9EvPOmX4kRm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--aa2bbbcd-20be-4619-a9d5-cd565c482dc4-0', usage_metadata={'input_tokens': 62, 'output_tokens': 78, 'total_tokens': 140, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser() #모델의 출력에서 문자열 내용만 추출하는 파서

result = model.invoke(messages)
parser.invoke(result) # AIMessage 객체가 아닌 순수 문자열 출력

'안녕하세요, 개스톤! 당신의 초대는 정말 감사하지만, 저는 야수와 함께 있는 이 집이 좋아요. 야수의 마음속에 숨겨진 아름다움이 있다는 것을 알게 되었답니다. 그와 함께하는 시간이 소중하거든요. 대신 다른 친구들과 즐거운 시간을 보내는 건 어떨까요?'

## 체인 생성

In [5]:
# invoke의 입력이 model로 전달되고, model의 출력이 자동으로 parser로 전달
chain = model | parser
chain.invoke(messages)

'안녕하세요, 개스톤. 당신이 이렇게 부르다니, 정말 놀라워요. 하지만 저녁은... 저에게는 조금 어려울 것 같아요. 저는 다른 분들과 함께하는 것을 더욱 소중히 생각하고 있거든요. 당신의 마음은 고맙지만, 우리 친구로서 지내면 좋겠어요. 언제든지 좋은 대화는 나눌 수 있으니, 함께 나누고 싶은 이야기가 있으면 말해줘요!'

## 프롬프트 템플릿
- {변수}를 넣어 쉽게 프롬프트를 만들 수 있음
- 딕셔너리 형태로 변수 값을 전달하면 완성된 프롬프트 생성

In [6]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "너는 {story}에 나오는 {character_a} 역할이다. 그 캐릭터에 맞게 사용자와 대화하라."
human_template = "안녕? 저는 {character_b}입니다. 오늘 시간 괜찮으시면 {activity} 같이 할까요?"

prompt_template = ChatPromptTemplate([
    ("system", system_template),
    ("user", human_template),
])

result = prompt_template.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

print(result)

messages=[SystemMessage(content='너는 미녀와 야수에 나오는 미녀 역할이다. 그 캐릭터에 맞게 사용자와 대화하라.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕? 저는 야수입니다. 오늘 시간 괜찮으시면 저녁 같이 할까요?', additional_kwargs={}, response_metadata={})]


## LCEL 체인 심화

입력 (딕셔너리) -> prompt_template

프롬프트 (메시지) -> model

모델 출력 (AIMessage) -> parser

최종 결과 (문자열)

In [7]:
chain = prompt_template | model | parser

chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

'안녕하세요, 야수님! 저녁을 함께 먹는 것은 정말 좋은 아이디어네요. 어떤 음식을 좋아하세요? 제가 준비할게요!'

## 체인 재사용

In [8]:
chain = prompt_template | model | parser

chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "개스톤",
    "activity": "저녁"
})

'안녕하세요, 개스톤. 제안해 주셔서 고마워요. 하지만 제가 저녁을 함께하는 것보다는 제 마음이 다른 곳에 있다는 것을 알아주셨으면 해요. 저는 진정한 사랑과 이해를 찾고 있답니다. 그렇지만 친구로서 함께 시간을 보내는 건 좋을 것 같아요!'

## 출력 구조화

### Pydantic 이란?

![이미지](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdna%2FbGANx3%2FbtrUuisjkwf%2FAAAAAAAAAAAAAAAAAAAAANsBUMSIh_Igw32SWHgX13XFytYN8IlVRpkRvRNn0_Vs%2Fimg.webp%3Fcredential%3DyqXZFxpELC7KVnFOS48ylbz2pIh7yKj8%26expires%3D1764514799%26allow_ip%3D%26allow_referer%3D%26signature%3DCu3WpMwN4tLwSv2cVQRNUdrTkcI%253D)

- 원하는 데이터의 스키마(구조)를 파이썬 클래스로 정의하게 해주는 라이브러리
- 런타임 타입 힌트 강제화
- 데이터 직렬화 역직렬화에 사용
- `BaseModel` : Pydantic 라이브러리에서 제공하는 기본 모델 클래스
  - 이 클래스를 상속하여 데이터 모델을 정의하고, 해당 모델의 필드와 유효성 검사 규칙을 설정할 수 있음
- ` Field` : 모델 필드를 정의할 때 사용
  - 필드를 통해 데이터 유효성 검사, 기본값 설정, 제약 조건 등을 세밀하게 제어

In [9]:
from typing import Literal
from pydantic import BaseModel, Field

class Adlib(BaseModel):
    """스토리 설정과 사용자 입력에 반응하는 대사를 만드는 클래스"""
    answer: str = Field(description="스토리 설정과 사용자와의 대화 기록에 따라 생성된 대사")
    main_emotion: Literal["기쁨", "분노", "슬픔", "공포", "냉소", "불쾌", "중립"] = Field(description="대사의 주요 감정")
    main_emotion_intensity: float = Field(description="대사의 주요 감정의 강도 (0.0 ~ 1.0)")

structured_llm = model.with_structured_output(Adlib) # 모델에게 Adlib Pydantic 클래스의 구조를 반드시 따르도록 설정
adlib_chain = prompt_template | structured_llm

adlib_chain.invoke({
    "story": "미녀와 야수",
    "character_a": "벨",
    "character_b": "개스톤",
    "activity": "저녁"
})

Adlib(answer='안녕하세요, 개스톤. 하지만 제가 당신과 저녁을 같이 하는 건 좀 어려울 것 같아요. 나는 책을 읽는 것을 더 좋아하고, 당신처럼 외향적인 상황이 좀 피곤하게 느껴지거든요. 그리고... 저는 다른 사람들의 마음을 이해하고 싶어요. 그런 점에서 당신과는 많이 다르네요.', main_emotion='불쾌', main_emotion_intensity=0.6)

### 문제 1)
BaseModel과 model.with_structured_output()을 함께 사용하는 이유를 적어보세요

- BaseModel은 데이터를 확인한 후에 유효성 검사 및 데이터 직렬화와 역직렬화를 사용한 후에 model.with_structured_output()이 BaseModel에서 유효성 검사 및 직렬화와 역직렬화가 맞춘 후 나온 데이터를 자동으로 메소드를 나오게 끔 하는 것이라고 생각해서 BaseModel과 model.with_structured_ouput()을 같이 사용하는 것이 좋다고 생각합니다. 
 